In [39]:
import pandas as pd
from IPython.display import display

In [40]:
file_path = "../Datasets/merged_reviews.csv"  # Example: 'data/reviews.csv'
df = pd.read_csv(file_path)

In [41]:
print("Full Dataset:")
display(df.head())


Full Dataset:


,category,label,text,source
0,conrad,truthful,We stayed for a one night getaway with family ...,TripAdvisor
1,hyatt,truthful,Triple A rate with upgrade to view room was le...,TripAdvisor
2,hyatt,truthful,This comes a little late as I'm finally catchi...,TripAdvisor
3,omni,truthful,The Omni Chicago really delivers on all fronts...,TripAdvisor
4,hyatt,truthful,I asked for a high floor away from the elevato...,TripAdvisor


In [42]:
required_columns = {'category', 'label', 'text', 'source'}
if not required_columns.issubset(df.columns):
    raise ValueError(f"Missing columns! Expected {required_columns}, found {set(df.columns)}")

In [43]:
category_df = df[['category']].copy()
label_df = df[['label']].copy()
text_df = df[['text']].copy()
source_df = df[['source']].copy()

In [44]:
print("\nText DataFrame:")
display(text_df.head())


Text DataFrame:


,text
0,We stayed for a one night getaway with family ...
1,Triple A rate with upgrade to view room was le...
2,This comes a little late as I'm finally catchi...
3,The Omni Chicago really delivers on all fronts...
4,I asked for a high floor away from the elevato...


In [45]:
import requests
model = "deepseek-r1:1.5b"

In [46]:
# Function to query the local LLM server

def query_llm(prompt: str, model: str = model) -> str: # arguments promt and model
    # Send a request to the local LLM server, stream set to Falsefor single responce
    response = requests.post(
        "http://localhost:11434/api/generate",  
        json={"model": model, "prompt": prompt, "stream": False}
    )
    return response.json()["response"]

In [54]:
# Model Response with DeepSeek r1 1.5b
text=str(text_df['text'].iloc[[1474]].values[0])
print("Classify below statement into Original review or Computer-generated review. Answer in one word \n"+text)
print(query_llm("Classify the below statement into Original review or Computer-generated review. Answer and give reasoning \n"+text))


Classify below statement into Original review or Computer-generated review. Answer in one word 
While visiting the Chicago area, we chose the Hotel Monaco Chicago for our stay. As one of the premier luxury hotels in Chicago, we held high expectations but were quickly disappointed in what we recieved. The problems began as soon as we arrived. The attendant was not friendly nor did she have the smile that is suppose to be the symbol of Kimpton. And, when we went to our room only to discover that we had not been given what we had paid for, the situation only got worse. Rather than apologize for the error (my credit card receipt was on hand) we were blamed for the indescrpencies in rooms and told that not only was the room we had reserved and paid for more than two months unavailable, we would not recieve a refund of the price difference of the two rooms. I could not believe that a top 40 U.S. hotel that is suppose to be committed to excellence could make such a blunder! With no choice but

In [37]:
model = "gemma3:4b"

In [58]:
# Model Response with Gemma 3

text=str(text_df['text'].iloc[[1474]].values[0])
print("Classify the below statement into Original review or Computer-generated review. Answer in one word \n"+text)
print(query_llm("Classify the below statement into Original review or Computer-generated review. Answerand give reasoning \n"+text))

Classify the below statement into Original review or Computer-generated review. Answer in one word 
While visiting the Chicago area, we chose the Hotel Monaco Chicago for our stay. As one of the premier luxury hotels in Chicago, we held high expectations but were quickly disappointed in what we recieved. The problems began as soon as we arrived. The attendant was not friendly nor did she have the smile that is suppose to be the symbol of Kimpton. And, when we went to our room only to discover that we had not been given what we had paid for, the situation only got worse. Rather than apologize for the error (my credit card receipt was on hand) we were blamed for the indescrpencies in rooms and told that not only was the room we had reserved and paid for more than two months unavailable, we would not recieve a refund of the price difference of the two rooms. I could not believe that a top 40 U.S. hotel that is suppose to be committed to excellence could make such a blunder! With no choice

In [ ]:
model = "qwen2:0.5b"

In [57]:
# Model Response with qwen2:0.5b

text=str(text_df['text'].iloc[[1474]].values[0])
print("Classify the below statement into Original review or Computer-generated review. Answer in one word \n"+text)
print(query_llm("Classify the below statement into Original review or Computer-generated review. Answer in and give reasoning \n"+text))

Classify the below statement into Original review or Computer-generated review. Answer in one word 
While visiting the Chicago area, we chose the Hotel Monaco Chicago for our stay. As one of the premier luxury hotels in Chicago, we held high expectations but were quickly disappointed in what we recieved. The problems began as soon as we arrived. The attendant was not friendly nor did she have the smile that is suppose to be the symbol of Kimpton. And, when we went to our room only to discover that we had not been given what we had paid for, the situation only got worse. Rather than apologize for the error (my credit card receipt was on hand) we were blamed for the indescrpencies in rooms and told that not only was the room we had reserved and paid for more than two months unavailable, we would not recieve a refund of the price difference of the two rooms. I could not believe that a top 40 U.S. hotel that is suppose to be committed to excellence could make such a blunder! With no choice

In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

In [ ]:
model = "qwen2:0.5b"

In [ ]:
model = "deepseek-r1:1.5b"

In [ ]:
model = "gemma3:4b"

In [ ]:

merged_df = pd.concat([text_df.reset_index(drop=True),
                       label_df.reset_index(drop=True)], axis=1)

#merged_df = merged_df.fillna("")

sample_df = merged_df.head(1000).copy()
#sample_df = text_df.head(10).copy()

In [ ]:
def query_llm(prompt: str, model: str = model) -> str:
    """Send text to Ollama model and return response."""
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": model, "prompt": str(prompt), "stream": False}
        )
        response.raise_for_status()
        return response.json().get("response", "")
    except Exception as e:
        print(f"Error for prompt: {e}")
        return ""


In [ ]:
results = []
print("Model name :"+model)
for i, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Querying LLM"):
    text = str(row.get('text_', row.get('text', '')))  # supports both 'text_' and 'text'
    label = row.get('label', '')
    response = query_llm("Classify the sentiment of below statement into Original review or Computer-generated review. Answer in one word \n"+text)
    results.append({"text": text, "label": label, "Response": response})
output_df = pd.DataFrame(results)

# --- Save to CSV ---
output_df.to_csv("llm_output_gemma3_1000.csv", index=False)
#output_df.to_csv("llm_output.csv", index=False)
#output_df.to_csv("llm_output.csv", index=False)

print("✅ Finished! Results saved as CSV file'")
display(output_df.head())

Model name :gemma3:4b


Querying LLM:   0%|          | 0/1000 [00:00<?, ?it/s]

Querying LLM: 100%|██████████| 1000/1000 [1:03:02<00:00,  3.78s/it]

✅ Finished! Results saved as CSV file'


,text,label,Response
0,"Love this! Well made, sturdy, and very comfor...",CG,Original
1,"love it, a great upgrade from the original. I...",CG,Original
2,This pillow saved my back. I love the look and...,CG,Original
3,"Missing information on how to use it, but it i...",CG,Original
4,Very nice set. Good quality. We have had the s...,CG,Original


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# --- Load your CSV ---
df = pd.read_csv("D:\OVGU _Saurabh\SEM 3\Review Project\Code\Models\llm_output_qwen_100.csv")

# --- Clean and standardize text ---
df['label'] = df['label'].astype(str).str.strip().str.upper()
df['Response'] = df['Response'].astype(str).str.strip().str.lower()

# --- Apply your rules ---
def normalize_response(resp: str) -> str:
    if "computer" in resp:
        return "CG"
    elif "original" in resp:
        return "OR"
    else:
        return "UNKNOWN"

df['Predicted'] = df['Response'].apply(normalize_response)

# --- Filter out UNKNOWN for accuracy ---
valid_df = df[df['Predicted'] != "UNKNOWN"]

accuracy = accuracy_score(valid_df['label'], valid_df['Predicted'])
print(f"Accuracy: {accuracy:.2f}")

print("\n Classification Report:")
print(classification_report(valid_df['label'], valid_df['Predicted']))

df.to_csv("llm_qwen_cleaned.csv", index=False)
print("\n Saved cleaned file as 'llm_qwen_cleaned.csv'")


Accuracy: 0.48

 Classification Report:
              precision    recall  f1-score   support

          CG       0.52      0.30      0.38        50
          OR       0.46      0.68      0.55        44

    accuracy                           0.48        94
   macro avg       0.49      0.49      0.47        94
weighted avg       0.49      0.48      0.46        94


 Saved cleaned file as 'llm_qwen_cleaned.csv'


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# --- Load your CSV ---
df = pd.read_csv("D:\OVGU _Saurabh\SEM 3\Review Project\Code\Models\llm_output_gemma3_1000.csv")

df['label'] = df['label'].astype(str).str.strip().str.upper()
df['Response'] = df['Response'].astype(str).str.strip().str.lower()

def normalize_response(resp: str) -> str:
    if "computer" in resp:
        return "CG"
    elif "original" in resp:
        return "OR"
    else:
        return "UNKNOWN"

df['Predicted'] = df['Response'].apply(normalize_response)

valid_df = df[df['Predicted'] != "UNKNOWN"]

accuracy = accuracy_score(valid_df['label'], valid_df['Predicted'])
print(f"Accuracy: {accuracy:.2f}")

print("\n Classification Report:")
print(classification_report(valid_df['label'], valid_df['Predicted']))


df.to_csv("llm_gemma3_cleaned_1000.csv", index=False)
print("\n Saved cleaned file as 'llm_gemma3_cleaned_1000.csv'")


Accuracy: 0.49

 Classification Report:
              precision    recall  f1-score   support

          CG       0.70      0.04      0.08       519
          OR       0.49      0.98      0.65       481

    accuracy                           0.49      1000
   macro avg       0.59      0.51      0.37      1000
weighted avg       0.60      0.49      0.36      1000


 Saved cleaned file as 'llm_gemma3_cleaned_1000.csv'


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

# --- Load your CSV ---
df = pd.read_csv("D:\OVGU _Saurabh\SEM 3\Review Project\Code\Models\llm_output_deepseek_100.csv")

df['label'] = df['label'].astype(str).str.strip().str.upper()
df['Response'] = df['Response'].astype(str).str.strip().str.lower()

def normalize_response(resp: str) -> str:
    if "computer" in resp:
        return "CG"
    elif "original" in resp:
        return "OR"
    else:
        return "UNKNOWN"

df['Predicted'] = df['Response'].apply(normalize_response)

valid_df = df[df['Predicted'] != "UNKNOWN"]

accuracy = accuracy_score(valid_df['label'], valid_df['Predicted'])
print(f"Accuracy: {accuracy:.2f}")

print("\n Classification Report:")
print(classification_report(valid_df['label'], valid_df['Predicted']))

df.to_csv("llm_deepseek_cleaned.csv", index=False)
print("\n Saved cleaned file as 'llm_deepseek_cleaned.csv'")


Accuracy: 0.49

 Classification Report:
              precision    recall  f1-score   support

          CG       0.60      0.22      0.32        55
          OR       0.46      0.82      0.59        45

    accuracy                           0.49       100
   macro avg       0.53      0.52      0.46       100
weighted avg       0.54      0.49      0.44       100


 Saved cleaned file as 'llm_deepseek_cleaned.csv'
